## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`


### Дополнительная часть (необязательная)


Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`

In [2]:
from requests import request
from bs4 import BeautifulSoup, ResultSet, Tag
import pandas
import time

In [3]:
def get_habr_preview_posts() -> ResultSet:
    index = 1
    results: ResultSet = []

    while True:
        response = request(
            method='GET',
            url='https://habr.com/ru/all/page{}/'.format(index)
        )
        soup = BeautifulSoup(response.text)
        posts = soup.find_all('article', 'post post_preview')

        if len(posts) == 0:
            break
        results += posts
        index += 1
        time.sleep(0.2)

    return results

preview_posts = get_habr_preview_posts()

In [6]:
def get_habr_posts(preview_posts: ResultSet) -> {str: Tag}:
    '''
    Getting full posts.

    :return: a dictionary with url and article data
    '''
    posts = dict()
    
    for post in preview_posts:
        post_title_link: Tag = post.find('a', class_='post__title_link')
        url = post_title_link.get('href')
        response = request('GET', url)
        soup = BeautifulSoup(response.text)
        find_result = soup.find_all('div', class_='content_left js-content_left')

        posts.setdefault(url, find_result[0])

    return posts

posts = get_habr_posts(preview_posts)

In [7]:
len(posts)

997

In [8]:
def filter_posts(
        dict_posts: {str: Tag},
        filter_words: [str]
) -> {str: ResultSet}:
    '''
    Filters articles by text considering filter_words
    '''
    result = dict()
    attribute_names = [
        'post__text post__text-html post__text_v1',
        'post__text post__text-html post__text_v2',
        'post__text post__text-html post__text_v3',
        'post__text post__text-html post__text_v4'
    ]
    for url in dict_posts:
        tag = None

        post: ResultSet = dict_posts[url]

        for name in attribute_names:
            tag = post .find('div', attrs={'class': name})

            if not tag is None:
                break

        if tag is None:
            continue

        for filter_word in filter_words:
            if not filter_word.lower() in tag.text.lower():
                continue
            result.setdefault(url, post)
            break
    return result

def transform_to_dataframe(posts: {str: Tag}) -> pandas.DataFrame:
    urls = []
    headers =[]
    date = []

    for url in posts:
        tag: Tag = posts[url].find('span', class_='post__title-text')
        time: Tag = posts[url].find('span', class_='post__time')

        headers.append(tag.text)
        urls.append(url)
        date.append(time.attrs['data-time_published'])

    return pandas.DataFrame({
        'date': date,
        'header': headers,
        'url': urls
    })

filter_habr_posts = filter_posts(
    dict_posts=posts,
    filter_words=['python', 'парсинг']
)
dataFrame = transform_to_dataframe(filter_habr_posts)

dataFrame

,date,header,url
0,2021-03-26T15:21Z,Поддержка токенов PKCS#11 с ГОСТ-криптографией...,https://habr.com/ru/post/549198/
1,2021-03-26T10:50Z,Google Earth Engine (GEE) как общедоступный ка...,https://habr.com/ru/post/549142/
2,2021-03-26T04:57Z,Pulsar vs Kafka: сравнение и мифы,https://habr.com/ru/company/southbridge/blog/5...
3,2021-03-24T15:00Z,Работаем с lightsquid или как сделать индивиду...,https://habr.com/ru/post/548782/
4,2021-03-24T13:24Z,Выводим нейронную сеть в продакшин для Android...,https://habr.com/ru/post/467055/
5,2021-03-24T10:32Z,Основы работы с Helm чартами и темплейтами — Ч...,https://habr.com/ru/post/548720/
6,2021-03-24T08:23Z,Полное руководство по созданию Docker-образа д...,https://habr.com/ru/company/mailru/blog/548480/
7,2021-03-23T14:19Z,Зачем мы транспилируем Haskell в JavaScript,https://habr.com/ru/company/typeable/blog/548574/
8,2021-03-22T23:01Z,CosmicPi: обнаружение космического излучения п...,https://habr.com/ru/company/selectel/blog/547872/
9,2021-03-22T18:41Z,"Привычный ужас в SIP, или о том, как не надо п...",https://habr.com/ru/post/543782/
